In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
from IPython.core.interactiveshell import InteractiveShell #multiple outputs for a cell
InteractiveShell.ast_node_interactivity = "all"            #multiple outputs for a cell
import os


In [ ]:
#reading the multiple .csv files from the terminal in the given directory
!ls /home/thakur/cks/data/*.csv>csvfilelist.txt

with open("csvfilelist.txt") as f:
    data=f.read().splitlines()

data=data[::-1] #reverse the order of the files
data[:5]

In [ ]:
#returns d->ddmmyyyy and dd->dd/mm/yyyy for providing data index of of a day
def get_date(num):
    idx=data[num].index(".")
    d=data[num][idx-8:idx]
    dd=d[:2]+"/"+d[2:4]+"/"+d[4:]
    return d,dd
#date for 3 days
date_today=get_date(0)[1];date_yesterday=get_date(1)[1];date_ereyesterday=get_date(2)[1]
# date_today
# date_yesterday
# date_ereyesterday

In [ ]:
#reading the first three dataframes
df_today=pd.read_csv(data[0],skiprows=1)
df_yesterday=pd.read_csv(data[1],skiprows=1)
df_ereyesterday=pd.read_csv(data[2],skiprows=1)

#copy of the data frames
df_today_copy=df_today.copy(deep=False)
df_yesterday_copy=df_yesterday.copy(deep=False)
df_ereyesterday_copy=df_ereyesterday.copy(deep=False)

In [ ]:
#important columns in the given dataframe
#ere yesterday info

preere=df_ereyesterday.pre_market
dayere=df_ereyesterday.day
postere=df_ereyesterday.post_market
loserere=df_ereyesterday.loser
actere=df_ereyesterday.most_active

#yesterday info
preyes=df_yesterday.pre_market
dayyes=df_yesterday.day
postyes=df_yesterday.post_market
loseryes=df_yesterday.loser
actyes=df_yesterday.most_active

#today info
pretoday=df_today.pre_market
daytoday=df_today.day
posttoday=df_today.post_market
losertoday=df_today.loser
acttoday=df_today.most_active

#common 
today_pre_day=reduce(np.intersect1d,(pretoday,daytoday))
today_day_post=reduce(np.intersect1d,(daytoday,posttoday))
today_pre_day_post=reduce(np.intersect1d,(pretoday,daytoday,posttoday))
today_pre_day_yesterday_post=reduce(np.intersect1d,(pretoday,daytoday,postyes))
today_gainer_active=reduce(np.intersect1d,(acttoday,daytoday))

today_pre_yesterday_post=reduce(np.intersect1d,(postyes,pretoday))
today_gainer_yesterday_loser=reduce(np.intersect1d,(daytoday,loseryes))
#today_pre_day_yesterday_post

active3days=reduce(np.intersect1d,(actere,actyes,acttoday))
active2days=reduce(np.intersect1d,(actyes,acttoday))

loser3days=reduce(np.intersect1d,(losertoday,loseryes,loserere))
loser2days=reduce(np.intersect1d,(losertoday,loseryes))

#today's active gainers
today_gainer_active
                     

In [ ]:
#returns the data frame locating given ticker if exists, if resulting data frame is empty it is ignored
def find_where(df,test_ticker):
    df_test=df.where(df==test_ticker)
    df_test.dropna(inplace=True,how='all')
    if df_test.empty:return
    df_test.fillna('-',inplace=True)
    return df_test
    
#begin the index of dataframe from 1 instead of 0
def change_index(df):
    df.index=np.arange(1,len(df)+1)
    
#returns a column of a dataframe as a list
def return_column(df,col):
    return df[col]

#find the sum of the given data frame
#def get_col_sum()
#return_column(df_today,"volume").sum()

In [ ]:
#first n values in today, yesterday, and ereyesterday dataframe
change_index(df_today_copy);change_index(df_yesterday_copy);change_index(df_ereyesterday_copy)
n=10
fmt=20*"-"+"{}"+20*"-"
print(fmt.format("today"))
df_today_copy.head(n)
print(fmt.format("yesterday"))
df_yesterday_copy.head(n)
print(fmt.format("ereyeasterday"))

df_ereyesterday_copy.head(n)


In [ ]:
#dataframe with post-yesterday, pre-today,

In [ ]:
#%%script false --no-raise-error #skip this cell and no error output
#returns position of pre-market values in day and after hours

print("Pre-market position on day on {} \n".format(date_today))
for i in df_today_copy.pre_market:
    find_where(df_today_copy,i)


In [ ]:
#returns the data frame based on two common rows ranking

def common1(df1,col1,df2,col2,rank=50):
    xy,x_ind,y_ind=np.intersect1d(df1[col1][:rank],df2[col2][:rank],return_indices=True)
    d={'ticker':xy,col1:x_ind,col2:y_ind}
    df=pd.DataFrame(data=d)
    change_index(df)
    return df
def common2(df1,col1,df2,col2,basis="day"):
    com=common1(df1,col1,df2,col2)
    sorted_com=com.sort_values(by=[basis])
    change_index(sorted_com)
    return sorted_com

# returns the data frame based on the common element in three columns
def common3(df1,col1,df2,col2,df3,col3,basis="day"):
    com1=common1(df1,col1,df2,col2)
    com2=common1(df2,col2,df3,col3)
    sorted_com1=com1.sort_values(by=[basis])
    sorted_com2=com2.sort_values(by=[basis])
    return sorted_com1,sorted_com2

#today's pre, day and after hours ranking
def pre_day_after_today():
    df1,df2=common3(df_today,"pre_market",df_today,"day",df_today,"post_market")
    result=pd.merge(df1,df2,on=["ticker","day"],how="outer")
    result=result.fillna('-')

    return result

#today's pre, yesterday's post
def pre_today_post_yesterday():
    #print("Pre-Market today Post-Market yesterday\n")
    df1=common1(df_today,"pre_market",df_yesterday,"post_market")
    df1=df1.sort_values(by=["pre_market"])
    return df1

#today's gainer and active
def today_active_gainer():
    df1=common1(df_today,"day",df_yesterday,"most_active")
    df1=df1.sort_values(by=["day"])
    return df1

#most active in 3 days
# def active2days():
#     df1=common2(df_today,"most_active",df_yesterday,"most_active","day")
#     #result=pd.merge(df1,df2,on=["ticker","most_active"],how="outer")
#     #result=result.fillna('-')
#     return df1

def all_in_one():
    print("yesterday's post_market today's pre_market\n")
    print(pre_today_post_yesterday())
    print("\ntoday's pre day and after_hours\n")
    print(pre_day_after_today())
    print("\ntoday's active gainers\n")
    print(today_active_gainer())

In [ ]:
#returns the data frame based on two common rows ranking

def common1(df1,col1,df2,col2,rank=50):
    xy,x_ind,y_ind=np.intersect1d(df1[col1][:rank],df2[col2][:rank],return_indices=True)
    d={'ticker':xy,col1:x_ind,col2:y_ind}
    df=pd.DataFrame(data=d)
    change_index(df)
    return df
def common2(df1,col1,df2,col2,basis="day"):
    com=common1(df1,col1,df2,col2)
    sorted_com=com.sort_values(by=[basis])
    change_index(sorted_com)
    return sorted_com

# returns the data frame based on the common element in three columns
def common3(df1,col1,df2,col2,df3,col3,basis="day"):
    com1=common1(df1,col1,df2,col2)
    com2=common1(df2,col2,df3,col3)
    sorted_com1=com1.sort_values(by=[basis])
    sorted_com2=com2.sort_values(by=[basis])
    return sorted_com1,sorted_com2

#today's pre, day and after hours ranking
def pre_day_after_today():
    df1,df2=common3(df_today,"pre_market",df_today,"day",df_today,"post_market")
    result=pd.merge(df1,df2,on=["ticker","day"],how="outer")
    result=result.fillna('-')

    return result

#today's pre, yesterday's post
def pre_today_post_yesterday():
    #print("Pre-Market today Post-Market yesterday\n")
    df1=common1(df_today,"pre_market",df_yesterday,"post_market")
    df1=df1.sort_values(by=["pre_market"])
    return df1

#today's gainer and active
def today_active_gainer():
    df1=common1(df_today,"day",df_yesterday,"most_active")
    df1=df1.sort_values(by=["day"])
    return df1

#most active in 3 days
# def active2days():
#     df1=common2(df_today,"most_active",df_yesterday,"most_active","day")
#     #result=pd.merge(df1,df2,on=["ticker","most_active"],how="outer")
#     #result=result.fillna('-')
#     return df1

def all_in_one():
    print("yesterday's post_market today's pre_market\n")
    print(pre_today_post_yesterday())
    print("\ntoday's pre day and after_hours\n")
    print(pre_day_after_today())
    print("\ntoday's active gainers\n")
    print(today_active_gainer())

In [ ]:
print("Post-Market yesterday Pre-Market today \n")
df1=common1(df_yesterday_copy,"post_market",df_today_copy,"pre_market")
df1=df1.sort_values(by=["pre_market"])
df1.set_axis(["ticker", "post_market "+date_yesterday,"pre_market "+date_today],axis=1,inplace=True)
df1

In [ ]:
#data frame of actives in different days, also gives the position of any ticker in given data-frame
most_active_dict=dict()
dat_len=len(data)
for i in range(dat_len):
    dat=get_date(i)[0]
    #print(i,"\t",dat)
    df=pd.read_csv(data[i],skiprows=1)
    act=df.most_active
    #print(act)
    most_active_dict[dat]=list(act)
df_most_active=pd.DataFrame(data=most_active_dict)
#change_index(df_most_active)
rank=50
#df_most_active[:rank]
test_ticker='NIO'
#find_where(df_most_active,test_ticker)

In [ ]:
#%%script false --no-raise-error #skip this cell and no error output
#ticker='NIO'
def color_tick_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for given value
    strings, black otherwise.
    """
    color = 'red' if val == ticker else 'black'
    return 'color: %s' % color

#s = df_most_active.style.applymap(color_tick_red)
#s

In [ ]:
#returns a single data frame including parameter in different dates
def get_all(col):
    col_dict=dict()
    dat_len=len(data)
    for i in range(dat_len):
        dat=get_date(i)[0]
        df=pd.read_csv(data[i],skiprows=1)
        acti=df[col]
        col_dict[dat]=list(acti)
    df_col=pd.DataFrame(data=col_dict)
    return df_col
#change_index(df_most_active)
#rank=50
#df_most_active[:rank]
#test_ticker='NIO'
#find_where(df_most_active,test_ticker)
#get_all("volume")

In [ ]:
#%%script false --no-raise-error #skip this cell and no error output
#identify the given ticker in the dataframe and color it "red"
ticker='SPRT'
coll=["pre_market","day","post_market","loser","most_active"]
for i in coll:
    print('{:>40}................{:}...................'.format(" ",i.upper()))
    all_df=get_all(i)
    all_df.to_csv("temp/test.txt",sep=" ",line_terminator='\n')
    s = all_df.style.applymap(color_tick_red)
    #s.to_csv("/temp/demosave.txt","\t")
    s

In [ ]:
%%script false --no-raise-error #skip this cell and no error output
#returns the dataframe of the given parameter
def get_dataframe(parameter):
        par_dict=dict()
        dat_len=len(data)
        for i in range(dat_len):
            dat=get_date(i)[0]
            df=pd.read_csv(data[i],skiprows=1)
            act=df[parameter]
            par_dict[dat]=list(act)
        df_par=pd.DataFrame(data=par_dict)
        return df_par
    
col=["pre_market","day","post_market","loser","most_active"]
today_gainers=df_today.day #gainers for today
#today_gainers
#today_gainers[1]
#ticker='ONTX' #change the ticker here
for ticker in today_gainers:
    print("ticker\t\"{}\"".format(ticker))
    for i in col:
        print(i)
    #get_dataframe(i)
        find_where(get_dataframe(i),ticker)
#get_dataframe("most_active")

In [ ]:
#getting the dataframe of most actives to create the plot
#test='IQ'
tick_no=50
#df_most_active=df_today.most_active
#df_most_active
test=df_most_active.iloc[:,0]#[:tick_no] #most active column for latest date
#test
#print(test)
value=[]
for i in range(len(test)):
    df_most=df_most_active.applymap(lambda x: True if x==test[i] else False)
    df_most_array=df_most.to_numpy()
    lenn=np.sum(df_most_array)
    #print("length\t",lenn)
    value.append(lenn)
#print(value)

In [ ]:
#changing the colors of the bar based on values
col=[]
for val in value:
    if val>=15:
        col.append('forestgreen')
    elif val>=10:
        col.append('brown')
    elif val>=5:
        col.append('thistle')
    else:
        col.append('silver')
#print(col)

In [ ]:
days=len(data)
days=days+1
plt.figure(figsize=(15,10))
plt.grid(axis='y',linestyle='--',linewidth=0.5,color='k')
ax=plt.axes()
ax.set_facecolor('oldlace')
plt.bar(test,value,width=0.8,tick_label=test,color=col)
plt.xticks(rotation=90)
plt.yticks(np.arange(1,days),[str(i) for i in range(1,days)])
plt.ylabel('active days')
plt.title('Active based on '+get_date(0)[1])
plt.xlabel('tickers')
plt.savefig('data/latestactive.pdf')
plt.show()

In [ ]:
#active data frame
from functools import reduce
df_act=get_all("day")
act_today=df_act[df_act.columns[0]]
act_yesterday=df_act[df_act.columns[1]]
act_ereyesterday=df_act[df_act.columns[2]]
#act_today,act_yesterday,act_ereyesterday
act_two_days=reduce(np.intersect1d,(act_today,act_yesterday))

print("active 2 days\t",act_two_days)
act_three_days=reduce(np.intersect1d,(act_today,act_yesterday,act_ereyesterday))
print("active 3 days\t",act_three_days)

In [ ]:
#get the volume vs date and price vs date plot for given list of tickers
#Using yfinance module to get the data from yahoo finance
from datetime import datetime
from datetime import timedelta
import pandas as pd
import matplotlib.pyplot as plt
import sys
import yfinance as yf
#import webbrowser
from openweb import open_web       #to open the webpages

#%matplotlib notebook
#yahoo history data
stock=input('Enter your stock:')
#stock=sys.argv[1]
ticker=yf.Ticker(stock)
 
fmtstr="{:-<100}\n{:>40}\n{:-<100}"
 
#history for the period
df=ticker.history(period='3mo')[::-1]
# valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
df.reset_index(inplace=True) #resetting the index of the date
df=df.drop(['Dividends', 'Stock Splits'],axis=1)
df['Volume']=df['Volume'].apply(lambda x:x/(10**6)) #volume to million
df['Date']=df['Date'].apply(lambda x:x.strftime('%d/%m/%y'))
print(fmtstr.format(" ","INFORMATION"," "))
print(df.to_string())
#print("Modified History",df)


def get_tick_data(ticker,dy=365):
    '''

    returns the data frame of the ticker 365 days ago (default) to me.
    '''
    end_date=datetime.today()
    diff=timedelta(days=dy)        #20 days difference
    start_date=(end_date-diff).strftime('%Y-%m-%d')
    #print("start_date\t",start_date)
    end_date=end_date.strftime('%Y-%m-%d')
    df = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)
    df.reset_index(inplace=True) #resetting the index of the date
    df['Volume']=df['Volume'].apply(lambda x:x/(10**6)) #volume to million
    df['Date']=df['Date'].apply(lambda x:x.strftime('%d/%m/%y'))
    df=df[::-1] #reverse
    return df

#plotting the parameter against the parameter
def par_vs_date(df,par1="Volume",par2="Close",days=30):
    """
    returs the plot of parameter vs date in given datafile
    """
    fig,(ax1,ax2)=plt.subplots(2,figsize=(20,16),sharex=True)
    
    #first plot
    bar1=ax1.bar(df["Date"][:days],df[par1][:days],alpha=0.4)
    #print(bars)
    for bar in bar1:
        #print("height\n",bar.get_height())
        h=bar.get_height();w=bar.get_width();x=bar.get_x();hr=round(h,2)
        ax1.text(x+w/2-0.2,h*1.01,hr,rotation=90,color='darkviolet',fontsize=12)
    #plt.figure(figsize=(10,8))
    par_mean=df[par1][-days:].mean()
    ax1.axhline(par_mean,color='r',linestyle='--',alpha=0.5)
    ax1.text(-1.5,par_mean,str(round(par_mean,1)),bbox=dict(facecolor='orange', alpha=0.5),fontsize=12)
    
    #plt.bar_label(df["date"][-days:],df[parameter][-days:])
    #ax.set_xticks(rotation=90)
    ax1.patch.set_facecolor('snow')
    ax1.set_ylabel(par1)
    #ax1.set_xlabel('date')
    #ax1.tick_params('x',labelrotation=90)
    #ax1.set_title(ticker)
    
    #second plot
    bar2=ax2.bar(df["Date"][:days],df[par2][:days],alpha=0.4,color='salmon')
    #print(bars)
    for bar in bar2:
        #print("height\n",bar.get_height())
        h=bar.get_height();w=bar.get_width();x=bar.get_x();hr=round(h,2)
        ax2.text(x+w/2-0.2,h*1.01,hr,rotation=90,color='blue',fontsize=12)
    #plt.figure(figsize=(10,8))
    par_mean2=df[par2][:days].mean()
    ax2.axhline(par_mean2,color='r',linestyle='--',alpha=0.5)
    ax2.text(-1.5,par_mean2,str(round(par_mean2,1)),bbox=dict(facecolor='orange', alpha=0.5),fontsize=12)
    
    #plt.bar_label(df["date"][-days:],df[parameter][-days:])
    #ax.set_xticks(rotation=90)
    ax2.patch.set_facecolor('snow')
    ax2.xaxis.set_label_position('top')
    ax2.set_ylabel(par2)
    ax2.set_xlabel('date')
    ax2.xaxis.tick_top()
    #ax2.tick_params('x',labelrotation=90)
    ax2.tick_params(axis='x',labelsize=12,labelbottom=False,labeltop=True,labelrotation=90,pad=15,colors='green',which='both')
    #ax2.axis.xticks(fontsize=10)
    
    #ax2.set_title(ticker)
    #plt.grid(true)
    fig.suptitle(stock.upper())
    plt.tight_layout()
    plt.savefig("plot/"+stock+'.pdf')
    
    plt.show()

#get information of ticker 
#ticker=input('enter the ticker: ')
#ticker='roll'
#df=get_tick_data(ticker)
#print(df.head(10))
#parameter_vs_date1('volume',df)
#parameter_vs_date1("volume",df)
#par_vs_date(df)
#print(df.head(40))


#open the webbrowsers
#open_web(stock) #function in the openweb.py

#plt.plot(df.date,df.close)
par_vs_date(df,days=60)




In [ ]:
df_act

In [ ]:
#Find the sum of top 50 active volumes on each day and plot against the day
day=[]
vol_sum=[]
j=0 #index counter
for i in data:
    df=pd.read_csv(i,skiprows=1)
    day.append(get_date(j)[1])
    j=j+1
    vol_sum.append(round(return_column(df,"volume").sum(),2))
#day
#vol_sum

#plot volume against day

plt.figure(figsize=(15,10))
plt.grid(axis='y',linestyle='--',linewidth=0.5,color='k')
ax=plt.axes()
plt.xticks(rotation=90)
ax.set_facecolor('oldlace')
plt.ylabel('volume (million)')
plt.title('Volume on different dates ')
plt.xlabel('date')
plt.bar(day,vol_sum,color='y')
plt.show()


In [ ]:
%%script false --no-raise-error #skip this cell and no error output
#gives when the given ticker was active
test='NVVE'
df_most_test=df_most_active.where(df_most_active==test,'-')
df_most_test


In [ ]:
#OUTPUT IN THIS TERMINAL
import datetime
print("printed on ",datetime.datetime.now())
dd=get_date(0)[1]
print(80*"-")
print("{:>40} ({})\n".format("DATE",dd))
print("\npre <=> day:\n\n{}".format(today_pre_day))
print("\nday <=> post:\n\n{}".format(today_day_post))
print("\npre <=> day <=>post:\n\n{}".format(today_pre_day_post))
print(80*"-")


print("\n{:>60}\n".format("TODAY'S PRE YESTERDAY'S AFTER HOUR"))
print("{}".format(today_pre_yesterday_post))
print(80*"-")

print("\n{:>60}\n".format("TODAY'S PRE, DAY,  YESTERDAY'S AFTER HOUR"))
print("{}\n".format(today_pre_day_yesterday_post))
print(80*"-")

#yesterday's loser and today's day
print("\n{:>60}\n".format("TODAY'S GAINER YESTERDAY'S LOSER"))
print("{}\n".format(today_gainer_yesterday_loser))
print(80*"-")

#most active and gainers today
print("\n{:>60}\n".format("TODAY'S GAINER AND MOST-ACTIVE"))
print("{}\n".format(today_gainer_active))
print(80*"-")

#active in 2 days
print("\n{:>60}\n".format("ACTIVE IN 2 DAYS"))
print("{}\n".format(active2days))
print(80*"-")

#active in 3 days
print("\n{:>60}\n".format("ACTIVE IN 3 DAYS"))
print("{}\n".format(active3days))
print(80*"-")

#loser in 2 days
print("\n{:>60}\n".format("LOSER IN 2 DAYS"))
print("{}\n".format(loser2days))
print(80*"-")

#loser in 3 days
print("\n{:>60}\n".format("LOSER IN 3 DAYS"))
print("{}\n".format(loser3days))
print(80*"-")

#volume in 3 days
print("\n{:>60}\n".format("VOLUME IN 3 DAYS"))

print("volume on {}:\t{:0.2f} millions".format(date_today,vol_sum[0]))
print("volume on {}:\t{:0.2f} millions".format(date_yesterday,vol_sum[1]))
print("volume on {}:\t{:0.2f} millions\n".format(date_ereyesterday,vol_sum[2]))
print(80*"-")

In [ ]:
%%capture cap --no-stderr
import datetime
print("printed on ",datetime.datetime.now())
dd=get_date(0)[1]
d=get_date(0)[0]
outputfile="data/output"+d+".txt"
with open(outputfile,'w') as out:
    out.write(cap.stdout)

print(80*"-")
print("{:>40} ({})\n".format("DATE",dd))
print("\npre <=> day:\n\n{}".format(today_pre_day))
print("\nday <=> post:\n\n{}".format(today_day_post))
print("\npre <=> day <=>post:\n\n{}".format(today_pre_day_post))
print(80*"-")


print("\n{:>60}\n".format("TODAY'S PRE YESTERDAY'S AFTER HOUR"))
print("{}".format(today_pre_yesterday_post))
print(80*"-")

print("\n{:>60}\n".format("TODAY'S PRE, DAY,  YESTERDAY'S AFTER HOUR"))
print("{}\n".format(today_pre_day_yesterday_post))
print(80*"-")

#yesterday's loser and today's day
print("\n{:>60}\n".format("TODAY'S GAINER YESTERDAY'S LOSER"))
print("{}\n".format(today_gainer_yesterday_loser))
print(80*"-")

#most active and gainers today
print("\n{:>60}\n".format("TODAY'S GAINER AND MOST-ACTIVE"))
print("{}\n".format(today_gainer_active))
print(80*"-")

#active in 2 days
print("\n{:>60}\n".format("ACTIVE IN 2 DAYS"))
print("{}\n".format(active2days))
print(80*"-")

#active in 3 days
print("\n{:>60}\n".format("ACTIVE IN 3 DAYS"))
print("{}\n".format(active3days))
print(80*"-")

#loser in 2 days
print("\n{:>60}\n".format("LOSER IN 2 DAYS"))
print("{}\n".format(loser2days))
print(80*"-")

#loser in 3 days
print("\n{:>60}\n".format("LOSER IN 3 DAYS"))
print("{}\n".format(loser3days))
print(80*"-")
#volume in 3 days
print("\n{:>60}\n".format("VOLUME IN 3 DAYS"))

print("volume on {}:\t{:0.2f} millions".format(date_today,vol_sum[0]))
print("volume on {}:\t{:0.2f} millions".format(date_yesterday,vol_sum[1]))
print("volume on {}:\t{:0.2f} millions\n".format(date_ereyesterday,vol_sum[2]))
print(80*"-")

In [ ]:
#clickable link for the intersted ticker
def make_clickable(val):
    return '<a target ="_blank" href="{}">{}</a>'.format(val,"link")
    #target="_blank" opens the window to new window
    #can use val inplace to "link" to print the exact link

In [ ]:
#active gainers
today_gainer_active
yahoo="https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch"
tradingview="https://www.tradingview.com/symbols/NASDAQ-{}/"
google="https://www.google.com/finance/quote/{}:NASDAQ"
finviz="https://finviz.com/quote.ashx?t={}"
webull='https://www.webull.com/quote/nasdaq-{}'
#print(link.format(ticker,ticker))
for ticker in today_gainer_active:
    print("\n..........."+ticker+"..............\n")
    print(yahoo.format(ticker,ticker))
    print(tradingview.format(ticker))
    print(google.format(ticker))
    print(finviz.format(ticker))
    print(webull.format(ticker))


#create the list links for link
#links=[link.format(ticker,ticker) for ticker in today_gainer_active]
#links
#trading_view_links=
# for i in today_gainer_active:
#     print(link.format(i,i))
#     print(i)
#df=pd.DataFrame(today_active_gainer,columns=['active'])

In [ ]:
yahoo_link=yahoo.format(ticker,ticker)
tradingview_link=tradingview.format(ticker)
google_link=google.format(ticker)
finviz_link=finviz.format(ticker)
webull_link=webull.format(ticker)

yahoo_link
tradingview_link
google_link
finviz_link
webull_link

In [ ]:
test_dict=list(today_gainer_active)
link_dict={"ticker":test,"yahoo":yahoo_link,"google":google_link,"tradingview":tradingview_link,"finviz":finviz,"webull":webull_link}
test_dict

In [ ]:
df=pd.DataFrame(test_dict)
df=df.style.format({'yahoo-link':make_clickable})


In [ ]:
df